In [ ]:
!pip install requests
import requests

In [ ]:
geo = requests.get('https://api.openweathermap.org/geo/1.0/direct?q=London,GB&limit=1&appid=34aab9a14e3edd9bcfd30e776f165843')
print(response.status_code)

NameError: name 'response' is not defined

In [ ]:
data = response.json()
print(data)
lat = geo.json()[0]['lat']
lon = geo.json()[0]['lon']
print(lat)
print(lon)

In [ ]:
pollution = requests.get(f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid=34aab9a14e3edd9bcfd30e776f165843")
print(response.status_code)

In [ ]:
print(pollution.json())

Open weather API Test

In [ ]:
import time
import pandas as pd

all_pollution_data = []

# The current OpenWeatherMap air pollution API provides current data, not historical ranges.
# This loop will make multiple requests for the *current* data,
# with a delay between requests. If you need historical data,
# you would need to use a different API or service.
for i in range(6):
   # The API does not support historical ranges with this endpoint
   # The f-string constructs the URL for the current air pollution data
   url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid=34aab9a14e3edd9bcfd30e776f165843"

   # Make the GET request directly using the URL string
   res = requests.get(url)

   # Check if the request was successful
   if res.status_code == 200:
       data = res.json().get("list", [])
       # Since the API returns current data, the 'list' will likely contain one entry per request.
       all_pollution_data.extend(data)
       print(f"Fetched segment {i+1} with {len(data)} record(s)")
   else:
       print(f"Failed to fetch data for segment {i+1}. Status code: {res.status_code}")

   time.sleep(1)  # be nice to the API, small delay

# Step 3: convert to DataFrame
records = []
for entry in all_pollution_data:
    # Ensure 'dt' key exists in the entry
    if 'dt' in entry:
        dt = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(entry['dt']))
        record = {
            "datetime": dt,
            **entry.get('components', {}), # Use .get() with empty dict as default
            "aqi": entry.get('main', {}).get('aqi') # Use .get() for nested access
        }
        records.append(record)
    else:
        print("Skipping entry due to missing 'dt' key:", entry)


df = pd.DataFrame(records)
print(df.head())
print(len(df), "total records collected")

In [ ]:
import requests
import pandas as pd

lat, lon = 51.5072, -0.1276  # London

# Split the year into months to avoid giant payloads
frames = []
for month in range(1, 13):
    start = f"2024-{month:02d}-01"
    # rough end of month: next month’s 1st minus 1 day
    end_month = month + 1 if month < 12 else 1
    end_year = 2024 if month < 12 else 2025
    end = f"{end_year}-{end_month:02d}-01"

    url = (
        f"https://air-quality-api.open-meteo.com/v1/air-quality?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,ozone,sulphur_dioxide"
        f"&start_date={start}&end_date={end}"
    )

    res = requests.get(url)
    data = res.json()
    df = pd.DataFrame(data["hourly"])
    frames.append(df)
    print(f"Fetched {start} → {end}, {len(df)} rows")

air_df = pd.concat(frames, ignore_index=True)
print(air_df.head())
print(len(air_df), "total hourly records")


In [ ]:
air_df.to_csv("air_quality_2024.csv", index=False)

In [ ]:
lat, lon = 51.5072, -0.1276  # London

# Split the year into months to avoid giant payloads
frames = []
for month in range(1, 13):
    start = f"2024-{month:02d}-01"
    # rough end of month: next month’s 1st minus 1 day
    end_month = month + 1 if month < 12 else 1
    end_year = 2024 if month < 12 else 2025
    end = f"{end_year}-{end_month:02d}-01"

   # Pollen
    pollen_url = (
        f"https://air-quality-api.open-meteo.com/v1/air-quality?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=alder_pollen,birch_pollen,grass_pollen,olive_pollen,ragweed_pollen"
        f"&start_date=2024-01-01&end_date=2025-01-01"
    )

    res = requests.get(url)
    data = res.json()
    df = pd.DataFrame(data["hourly"])
    frames.append(df)
    print(f"Fetched {start} → {end}, {len(df)} rows")

pollen_url_df = pd.concat(frames, ignore_index=True)
print(df.head())
print(len(df), "total hourly records")

**Open meteo API calls


In [ ]:
import requests
import pandas as pd
import time

# -----------------------------
# CONFIG
# -----------------------------
lat, lon = 51.5072, -0.1276   # London coordinates
year = 2024

# -----------------------------
# 1️⃣ AIR QUALITY (Hourly, Full Year)
# -----------------------------
air_frames = []

for month in range(1, 13):
    start = f"{year}-{month:02d}-01"
    end_month = month + 1 if month < 12 else 1
    end_year = year if month < 12 else year + 1
    end = f"{end_year}-{end_month:02d}-01"

    air_url = (
        f"https://air-quality-api.open-meteo.com/v1/air-quality?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,ozone,sulphur_dioxide"
        f"&start_date={start}&end_date={end}"
    )

    res = requests.get(air_url)
    data = res.json()
    df = pd.DataFrame(data["hourly"])
    air_frames.append(df)
    print(f"Fetched AIR {start} → {end} ({len(df)} rows)")
    time.sleep(1)

air_df = pd.concat(air_frames, ignore_index=True)
air_df.to_csv("air_quality_2024.csv", index=False)
print(f"\n✅ Air Quality: {len(air_df)} hourly records saved\n")

# -----------------------------
# 2️⃣ POLLEN (Hourly, Seasonal)
# -----------------------------
pollen_frames = []

for month in range(1, 13):
    start = f"{year}-{month:02d}-01"
    end_month = month + 1 if month < 12 else 1
    end_year = year if month < 12 else year + 1
    end = f"{end_year}-{end_month:02d}-01"

    pollen_url = (
        f"https://air-quality-api.open-meteo.com/v1/air-quality?"
        f"latitude={lat}&longitude={lon}"
        f"&hourly=alder_pollen,birch_pollen,grass_pollen,olive_pollen,ragweed_pollen"
        f"&start_date={start}&end_date={end}"
    )

    res = requests.get(pollen_url)
    data = res.json()
    df = pd.DataFrame(data["hourly"])
    pollen_frames.append(df)
    print(f"Fetched POLLEN {start} → {end} ({len(df)} rows)")
    time.sleep(1)

pollen_df = pd.concat(pollen_frames, ignore_index=True)
pollen_df = pollen_df.fillna(0)   # Fill missing months (off-season)
pollen_df.to_csv("pollen_2024.csv", index=False)
print(f"\n✅ Pollen: {len(pollen_df)} hourly records saved\n")

# -----------------------------
# 3️⃣ WEATHER (Daily, Full Year)
# -----------------------------
weather_url = (
    f"https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={lat}&longitude={lon}"
    f"&start_date={year}-01-01&end_date={year+1}-01-01"
    f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,relative_humidity_2m_mean"
)

res = requests.get(weather_url)
res.raise_for_status()  # throws an error if the request fails

data = res.json()
weather_df = pd.DataFrame(data["daily"])
weather_df.to_csv("weather_2024.csv", index=False)
print(f"\n✅ Weather: {len(weather_df)} daily records saved\n")

In [ ]:
import requests
import pandas as pd
import time

lat, lon = 51.5072, -0.1276  # London
year = 2024

# Split into months to avoid huge payloads
weather_frames = []

for month in range(1, 13):
    start = f"{year}-{month:02d}-01"
    end_month = month + 1 if month < 12 else 1
    end_year = year if month < 12 else year + 1
    end = f"{end_year}-{end_month:02d}-01"

    weather_url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={start}&end_date={end}"
        f"&hourly=temperature_2m,relative_humidity_2m,precipitation,dew_point_2m,wind_speed_10m"
    )

    res = requests.get(weather_url)
    res.raise_for_status()
    data = res.json()

    # Make sure the data is valid
    if "hourly" in data:
        df = pd.DataFrame(data["hourly"])
        weather_frames.append(df)
        print(f"Fetched WEATHER {start} → {end} ({len(df)} rows)")
    else:
        print(f"⚠️ No hourly weather data for {start} → {end}")

    time.sleep(1)

weather_df = pd.concat(weather_frames, ignore_index=True)
weather_df.to_csv("weather_hourly_2024.csv", index=False)

print(f"\n✅ Weather (Hourly): {len(weather_df)} rows saved successfully!\n")

# **  Air Quality data   AQA**
A total of 9072 records of hourly data were extracted from [Open Mateo Air Quality API](https://open-meteo.com/en/docs/air-quality-api?timezone=GMT&time_mode=time_interval&start_date=2023-01-13&end_date=2024-01-01). from the year 2024 data The following was collected:

*   time
*   pm10
*   pm2_5
*   carbon_monoxide
*   nitrogen_dioxide  
*   ozone
*   sulphur_dioxide










In [ ]:
aqa_df = pd.read_csv("air_quality_2024.csv")
print(aqa_df.head())
print(len(aqa_df), "total records collected")

In [ ]:
aqa_df.info()

# **Pollen data**
A total of 9072 records of hourly data were extracted from [Open Mateo Pollen API](https://open-meteo.com/en/docs/air-quality-api?timezone=GMT&time_mode=time_interval&start_date=2023-01-13&end_date=2024-01-01).filtered from the year 2024  The following was collected:

*   time
*   alder_pollen
*   birch_pollen
*   grass_pollen
*   olive_pollen  
*   rageweed_pollen



In [ ]:
pollen_df = pd.read_csv("pollen_2024.csv")
print(pollen_df.head())
print(len(pollen_df), "total records collected")

In [ ]:
pollen_df.describe()

Due to pollen being highly seasonal majority of rows outside of spring/summer will likely be zero. It has been decided for accuracy of data to be left alone as it simulates accurate real life results

# ** Weather data*
A total of 9072 records of hourly data were extracted from [Open Mateo Air Quality API](https://open-meteo.com/en/docs/air-quality-api?timezone=GMT&time_mode=time_interval&start_date=2023-01-13&end_date=2024-01-01). from the year 2024 The following was collected:

*   time
*   temperature_2m
*   relative_humidity_2m
*   precipitation
*   dew_point_2m      
*   wind_speed_10m



In [ ]:
weather_df = pd.read_csv("weather_hourly_2024.csv")
print(weather_df.head())
print(len(weather_df), "total records collected")

In [ ]:
weather_df.info()


# **Merging environmental data**
in this section Environmental data will be merged to form env_df


In [ ]:
env_df = pd.merge(aqa_df, pollen_df, on="time", how="inner")
env_df = pd.merge(env_df, weather_df, on="time", how="inner")
print(env_df.head())

In [ ]:
env_df.to_csv("env_df.csv", index=False)

In [ ]:
env_df.info()

# **Asthma incident data**

A total of 217528 total  records were extract from [Public Health England Fingertips API](https://fingertips.phe.org.uk/profile/guidance/supporting-information/api). Which then was filtered for the year 2023/24. This resulted in 33578 records being retained for analysis from the following Asthma related indicators.

*   Hospital admissions for asthma (under 19 years) (9774 records)
*   Asthma: QOF prevalence (8221)
*   Patients with Asthma: review in the last 12 months (denominator incl PCAs) (7723)
*   Patients with Asthma (6-19 yrs): Second-hand smoking status recorded in the last 12 months (denominator incl. PCAs)	(7703)
*   Emergency hospital admissions for asthma in adults (aged 19 years and over)	(157)

The following code snippet demonstrates how data for the selected asthma indicators was retrieved using the Fingertips API




In [ ]:
import requests
import pandas as pd
!pip install fingertips-py

import fingertips_py as ftp
import pandas as pd

# Indicator ID for "Hospital admissions for asthma (under 19 years)"
indicator_id = 90810

# Fetch data for all available geographies
df = ftp.get_data_for_indicator_at_all_available_geographies(indicator_id)

print(df.head())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   Indicator ID                                   Indicator Name Parent Code  \
0         90810  Hospital admissions for asthma (under 19 years)         NaN   
1         90810  Hospital admissions for asthma (under 19 years)         NaN   
2         90810  Hospital admissions for asthma (under 19 years)         NaN   
3         90810  Hospital admissions for asthma (under 19 years)         NaN   
4         90810  Hospital admissions for asthma (under 19 years)         NaN   

  Parent Name  Area Code Area Name Area Type   Sex       Age  \
0         NaN  E92000001   England   England  Male  0-18 yrs   
1         NaN  E92000001   England   England  Male  0-18 yrs   
2         NaN  E92000001   England   England  Male  0-18 yrs   
3         NaN  E92000001   England   England  Male  0-18 yrs   
4         NaN  E92000001   England   England  Male  0-18 yrs   

                                       Category Type  

In [ ]:
cols = [
    "Indicator ID", "Indicator Name", "Area Code", "Area Name",
    "Sex", "Age", "Time period", "Value"
]
df_clean = df[cols]
print(df_clean.head())

   Indicator ID                                   Indicator Name  Area Code  \
0         90810  Hospital admissions for asthma (under 19 years)  E92000001   
1         90810  Hospital admissions for asthma (under 19 years)  E92000001   
2         90810  Hospital admissions for asthma (under 19 years)  E92000001   
3         90810  Hospital admissions for asthma (under 19 years)  E92000001   
4         90810  Hospital admissions for asthma (under 19 years)  E92000001   

  Area Name   Sex       Age Time period       Value  
0   England  Male  0-18 yrs     2013/14  232.464199  
1   England  Male  0-18 yrs     2013/14  384.660000  
2   England  Male  0-18 yrs     2013/14  275.740000  
3   England  Male  0-18 yrs     2013/14  235.300000  
4   England  Male  0-18 yrs     2013/14  237.970000  


In [ ]:
import fingertips_py as ftp
import pandas as pd
import time

indicator_ids = [90933, 92780, 93790, 93791, 93644, 93573, 93594, 93595, 90810]
all_data = []

for ind in indicator_ids:
    print(f"Fetching {ind} ...")
    try:
        df_temp = ftp.get_data_for_indicator_at_all_available_geographies(ind)
        df_temp["Indicator ID"] = ind
        all_data.append(df_temp)
        time.sleep(1)
    except Exception as e:
        print(f"⚠️ Failed for {ind}: {e}")

df = pd.concat(all_data, ignore_index=True)
print(df[["Indicator ID", "Indicator Name", "Area Name", "Time period", "Value"]].head())

Fetching 90933 ...
Fetching 92780 ...
Fetching 93790 ...
Fetching 93791 ...
Fetching 93644 ...


KeyboardInterrupt: 

In [ ]:
df.to_csv("asthma_full_dataset.csv", index=False)

# **Asthma dataset 2023/24 and cleaning**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84890 entries, 0 to 4190
Data columns (total 27 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Indicator ID                              84890 non-null  int64  
 1   Indicator Name                            84890 non-null  object 
 2   Parent Code                               83240 non-null  object 
 3   Parent Name                               83240 non-null  object 
 4   Area Code                                 84890 non-null  object 
 5   Area Name                                 84890 non-null  object 
 6   Area Type                                 84890 non-null  object 
 7   Sex                                       84890 non-null  object 
 8   Age                                       84890 non-null  object 
 9   Category Type                             3102 non-null   object 
 10  Category                                

In [ ]:
df['Time period'].unique()

array(['2013/14', '2014/15', '2015/16', '2016/17', '2017/18', '2018/19',
       '2019/20', '2020/21', '2021/22', '2022/23', '2023/24'],
      dtype=object)

# **Air quality affect in 2025 affect on data choices**

Due to the time period between 2023 and 2024 being most recent data of Asthma incidents data it has been chosen for ML data. Though is a year prior too environmental data it will still provide the most accurate estimate in the chang in asthma case compared to using environmental data yr prior due to increase in asthma in the past year due to air pollution levels after latest statistics reveal that the number of asthma attacks being treated in general practice have increased 45% in a year. [The Royal College of GPs](https://www.rcgp.org.uk/news/asthma-attacks-rise-dramatically)


In [ ]:
df_2023_24 = df[df["Time period"].astype(str).str.contains("2023/24", na=False)]
print(df_2023_24.head())
print(len(df_2023_24), "total records collected for 2023/24")

       Indicator ID                                   Indicator Name  \
16620         90810  Hospital admissions for asthma (under 19 years)   
16621         90810  Hospital admissions for asthma (under 19 years)   
16622         90810  Hospital admissions for asthma (under 19 years)   
16623         90810  Hospital admissions for asthma (under 19 years)   
16624         90810  Hospital admissions for asthma (under 19 years)   

      Parent Code Parent Name  Area Code Area Name Area Type   Sex       Age  \
16620         NaN         NaN  E92000001   England   England  Male  0-18 yrs   
16621         NaN         NaN  E92000001   England   England  Male  0-18 yrs   
16622         NaN         NaN  E92000001   England   England  Male  0-18 yrs   
16623         NaN         NaN  E92000001   England   England  Male  0-18 yrs   
16624         NaN         NaN  E92000001   England   England  Male  0-18 yrs   

                                           Category Type  ...    Count  \
16620       

In [ ]:
df_2023_24["Indicator Name"].value_counts()

,count
Indicator Name,
Hospital admissions for asthma (under 19 years),9774


In [ ]:
df_2023_24.to_csv("asthma_2023_24.csv", index=False)

In [ ]:
drop_columns =  []
keep_columns = [ "Value" , "Indicator Name"]
for col in df_2023_24.columns:
    if col not in keep_columns:
        drop_columns.append(col)
clean_asthma_df = df_2023_24.drop(columns = drop_columns)



# **Justification of dropping columns**

All columns Except Value and indicator name have been dropped due to being noise. Indicator name has been kept for stratified sampling

In [ ]:
clean_asthma_df.head()

,Indicator Name,Value
16620,Hospital admissions for asthma (under 19 years),171.373218
16621,Hospital admissions for asthma (under 19 years),248.105840
16622,Hospital admissions for asthma (under 19 years),238.302360
16623,Hospital admissions for asthma (under 19 years),150.599070
16624,Hospital admissions for asthma (under 19 years),209.976850


In [ ]:
clean_asthma_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9774 entries, 16620 to 4190
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator Name  9774 non-null   object 
 1   Value           8383 non-null   float64
dtypes: float64(1), object(1)
memory usage: 229.1+ KB


In [ ]:
clean_asthma_df.to_csv("clean_asthma_df.csv", index=False) #Saved pre sampling for repreducibility

# **Stratify Sampling justification**
Due to high incidents records of 32180  relative to training data too better reflect real world rarity of asthmetic Excerbations  [ERS Publications](https://https://publications.ersnet.org/content/erj/58/6/2100413) approximated 8-12% of patients with Asthma experience >= exacerbations per year.
Therefore for modellling purposes, the incidences will be sampled at 11% to represent the upper-midpoint of this established range.




In [ ]:
asthma_stratified = clean_asthma_df.groupby('Indicator Name', group_keys=False)\
    .apply(lambda x: x.sample(frac=0.11, random_state=42)) # random_state used for reproducibility

/tmp/ipython-input-3074632079.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=0.11, random_state=42))


In [ ]:
asthma_stratified.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1075 entries, 19080 to 17845
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator Name  1075 non-null   object 
 1   Value           912 non-null    float64
dtypes: float64(1), object(1)
memory usage: 25.2+ KB


In [ ]:
asthma_stratified = asthma_stratified.drop(columns = ["Indicator Name"]) # unnecessary now as data is stratified

In [ ]:
asthma_stratified.to_csv("asthma_stratified.csv", index= False)

In [ ]:
asthma_stratified.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1075 entries, 19080 to 17845
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   912 non-null    float64
dtypes: float64(1)
memory usage: 16.8 KB


In [ ]:
asthma_stratified.describe()

,Value
count,912.000000
mean,157.046082
std,79.005416
min,0.000000
25%,101.957586
50%,139.844693
75%,194.278498
max,559.159025


# **Getting training feature ready for ML (x)**

In [ ]:
env_df = pd.read_csv("env_df.csv")
env_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10656 entries, 0 to 10655
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   time                  10656 non-null  object 
 1   pm10                  10656 non-null  float64
 2   pm2_5                 10656 non-null  float64
 3   carbon_monoxide       10656 non-null  float64
 4   nitrogen_dioxide      10656 non-null  float64
 5   ozone                 10656 non-null  float64
 6   sulphur_dioxide       10656 non-null  float64
 7   alder_pollen          10656 non-null  float64
 8   birch_pollen          10656 non-null  float64
 9   grass_pollen          10656 non-null  float64
 10  olive_pollen          10656 non-null  float64
 11  ragweed_pollen        10656 non-null  float64
 12  temperature_2m        10656 non-null  float64
 13  relative_humidity_2m  10656 non-null  int64  
 14  precipitation         10656 non-null  float64
 15  dew_point_2m       

In [ ]:
env_df = env_df.drop(columns = ["time"])

In [ ]:
env_df.to_csv("env_df.csv", index = False)

# **Aligning and Standard Scaling X and Y**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
env_df = pd.read_csv('/content/drive/MyDrive/Asthma FYP /training data/raw training Data /env_df-2.csv')
asthma_stratified = pd.read_csv("/content/drive/MyDrive/Asthma FYP /training data/raw training Data /asthma_stratified.csv")

In [ ]:
env_sampled = env_df.sample(n=len(asthma_stratified), random_state=42).reset_index(drop=True)
y = asthma_stratified['Value'].reset_index(drop=True)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(env_sampled)

In [ ]:
y = asthma_stratified.dropna(subset=['Value']).reset_index(drop=True)

In [ ]:
y.to_csv("y.csv", index = False)

In [ ]:
X_scaled = X_scaled[:len(y)]

In [ ]:
X_scaled_df = pd.DataFrame(X_scaled)
X_scaled_df.to_csv("X_scaled.csv", index = False)


# **Training the split**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)